# <center> Sentiment analysis of Harry Potter 🪄⚡️ </center>

In [1]:
#!pip3 install nltk
#!pip3 install textblob


#!pip3 install spaCy
#!pip3 install gensim
#!pip3 install pattern

#!pip3 install speechrecognition

#!pip3 install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.2.0/en_core_web_sm-2.2.0.tar.gz

#!pip3 install speechrecognition
#!brew install portaudio
#!pip3 install pyaudio

In [2]:
from pymongo import MongoClient
import pandas as pd

In [3]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from textblob import TextBlob
import spacy
from spacy import displacy

In [47]:
import en_core_web_sm

In [5]:
import speech_recognition as sr 
import pyaudio

In [6]:
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/fernandocosta/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/fernandocosta/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fernandocosta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## 1. Bringing in the data

### 1.1. Connecting to the database

In [7]:
client = MongoClient("mongodb://localhost/HP") 
db = client.get_database()
collection = db['frases']

### 1.2. Dialogues

In [8]:
personajes = collection.distinct('character_name')
personajes[:3]

['Albus Dumbledore', 'Argus Filch', 'Bloody Baron']

In [9]:
query_voldemort = {"character_name":"Voldemort"}
voldemort = list(collection.find(query_voldemort, {"dialogue": 1,"_id": 0}))

In [10]:
voldemort

[{'dialogue': 'Use the boy.'},
 {'dialogue': 'He lies.'},
 {'dialogue': 'Let me speak to him.'},
 {'dialogue': 'I have strength enough for this. Harry Potter, we meet again.'},
 {'dialogue': "Yes, you see what I've become? See what I must do to survive? Live off another, a mere parasite. Unicorn blood can sustain me, but it cannot give me a body of my own. But there is something that can. Something that, conveniently enough, lies in your pocket. Stop him! Don't be a fool. Why suffer an horrific death, when you can join me, and live."},
 {'dialogue': "Haha. Bravery. Your parents had it too. Tell me, Harry, would you like to see your mother and father again, together? We can bring them back. All I ask is for something in return. That's it, Harry. There is no good and evil, there is only power, and those too weak to see it. Together, we'll do extraordinary things. Just give me the stone."},
 {'dialogue': 'Fool! Get the stone!'},
 {'dialogue': 'Kill him!'}]

# 2. Dialogues into a dataframe

### 2.1. Through mongo

In [11]:
voldemort_frases = [i['dialogue'] for i in voldemort]
voldemort_frases

['Use the boy.',
 'He lies.',
 'Let me speak to him.',
 'I have strength enough for this. Harry Potter, we meet again.',
 "Yes, you see what I've become? See what I must do to survive? Live off another, a mere parasite. Unicorn blood can sustain me, but it cannot give me a body of my own. But there is something that can. Something that, conveniently enough, lies in your pocket. Stop him! Don't be a fool. Why suffer an horrific death, when you can join me, and live.",
 "Haha. Bravery. Your parents had it too. Tell me, Harry, would you like to see your mother and father again, together? We can bring them back. All I ask is for something in return. That's it, Harry. There is no good and evil, there is only power, and those too weak to see it. Together, we'll do extraordinary things. Just give me the stone.",
 'Fool! Get the stone!',
 'Kill him!']

In [12]:
harrypotterdf = pd.DataFrame(voldemort_frases, columns=["Voldi"])
harrypotterdf.head()

,Voldi
0,Use the boy.
1,He lies.
2,Let me speak to him.
3,"I have strength enough for this. Harry Potter,..."
4,"Yes, you see what I've become? See what I must..."


#### 2.2. Through pandas

In [13]:
todo = pd.DataFrame(list(collection.find()))
todo.head()

,_id,scene,character_name,dialogue
0,5fbfc91a00dd010fcf9cb2ac,1,Minerva McGonagall,And the boy?
1,5fbfc91a00dd010fcf9cb2ad,1,Albus Dumbledore,"No problems, I trust, Hagrid?"
2,5fbfc91a00dd010fcf9cb2ae,1,Minerva McGonagall,"Good evening, Professor Dumbledore. Are the ru..."
3,5fbfc91a00dd010fcf9cb2af,1,Minerva McGonagall,Do you think it wise to trust Hagrid with some...
4,5fbfc91a00dd010fcf9cb2b0,1,Albus Dumbledore,"I'm afraid so, Professor. The good, and the bad."


In [14]:
todo = todo[['character_name', 'dialogue']]
todo.head()

,character_name,dialogue
0,Minerva McGonagall,And the boy?
1,Albus Dumbledore,"No problems, I trust, Hagrid?"
2,Minerva McGonagall,"Good evening, Professor Dumbledore. Are the ru..."
3,Minerva McGonagall,Do you think it wise to trust Hagrid with some...
4,Albus Dumbledore,"I'm afraid so, Professor. The good, and the bad."


In [15]:
todo.groupby(['character_name'])
todo

,character_name,dialogue
0,Minerva McGonagall,And the boy?
1,Albus Dumbledore,"No problems, I trust, Hagrid?"
2,Minerva McGonagall,"Good evening, Professor Dumbledore. Are the ru..."
3,Minerva McGonagall,Do you think it wise to trust Hagrid with some...
4,Albus Dumbledore,"I'm afraid so, Professor. The good, and the bad."
...,...,...
788,Harry Potter,You liar!
789,Harry Potter,"But Hagrid, we're not allowed to do magic away..."
790,Hermione Granger,"Feels strange to be going home, doesn't it?"
791,Albus Dumbledore,"Love, Harry, love. Ah. Bertie Bott's Every Fla..."


# 3. Sentiment analysis: how bad is the villain?

#### 3.1. NLTK

In [16]:
sia = SentimentIntensityAnalyzer()

In [17]:
sentence = "this is beautiful"

In [18]:
polarity = sia.polarity_scores(sentence)

In [19]:
polarity

{'neg': 0.0, 'neu': 0.339, 'pos': 0.661, 'compound': 0.5994}

In [20]:
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    return polarity

In [21]:
def sentimentAnalysis(sentence):
    sia = SentimentIntensityAnalyzer()
    polarity = sia.polarity_scores(sentence)
    pol = polarity['compound']
    return pol

In [22]:
todo['sentiment_compound'] = todo['dialogue'].apply(sentimentAnalysis)
todo.head()

,character_name,dialogue,sentiment_compound
0,Minerva McGonagall,And the boy?,0.0000
1,Albus Dumbledore,"No problems, I trust, Hagrid?",-0.1531
2,Minerva McGonagall,"Good evening, Professor Dumbledore. Are the ru...",0.6908
3,Minerva McGonagall,Do you think it wise to trust Hagrid with some...,0.8020
4,Albus Dumbledore,"I'm afraid so, Professor. The good, and the bad.",-0.0865


In [23]:
todo.groupby(['character_name'])['sentiment_compound'].mean()

character_name
Albus Dumbledore        0.348971
Argus Filch            -0.039550
Bloody Baron            0.000000
Class                   0.220200
Doris Crockford         0.000000
Draco Malfoy            0.027092
Dudley Dursley         -0.038871
Filius Flitwick         0.601500
Firenze                -0.151320
Fred Weasley            0.030260
Garrick Ollivander      0.559967
George Weasley         -0.131050
Ginny Weasley           0.709600
Goblin                  0.169200
Griphook                0.185800
Guard                   0.504000
Harry Potter           -0.003519
Hermione Granger        0.022553
Lee Jordan              0.102813
Man                    -0.658000
Marcus Flint            0.000000
Minerva McGonagall      0.213074
Molly Weasley           0.152780
Nearly Headless Nick    0.090580
Neville Longbottom     -0.065636
Oliver Wood             0.014775
Percy Weasley           0.381267
Petunia Dursley         0.153223
Quirinus Quirrell       0.124412
Rolanda Hooch           0.27

#### 3.2. TextBlob

In [24]:
text = "i hate it here"

In [25]:
blob = TextBlob(text)
blob.sentiment

Sentiment(polarity=-0.8, subjectivity=0.9)

In [26]:
en_blob = TextBlob(u'que tengas buen buen day')
en_blob=en_blob.translate(from_lang='es',to='en')

In [27]:
nl_blob = TextBlob(u'Ik heb in Nederlandse gewoond vor drie maanden')
hola = nl_blob.translate(to='en')
hola

TextBlob("I have lived in the Netherlands for three months")

In [28]:
def sentiment(text):
    en_blob=TextBlob(u'{}'.format(text))
    translated = en_blob.translate(to='en')
    return translated.sentiment

In [29]:
text = 'Ik heb in Nederlandse gewoond vor drie maanden'

In [30]:
sentiment(text)

Sentiment(polarity=0.0, subjectivity=0.0)

In [31]:
sentiment(text)

Sentiment(polarity=0.0, subjectivity=0.0)

## 4. Frequency

### 4.1. Tokenization

In [32]:
text = todo.loc[4, 'dialogue']
text

"I'm afraid so, Professor. The good, and the bad."

In [54]:
text="What's the best time of the year to learn Python?"
words = nltk.word_tokenize(text)
words

['What',
 "'s",
 'the',
 'best',
 'time',
 'of',
 'the',
 'year',
 'to',
 'learn',
 'Python',
 '?']

In [55]:
#Removing symbols

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(text)
tokens

['What',
 's',
 'the',
 'best',
 'time',
 'of',
 'the',
 'year',
 'to',
 'learn',
 'Python']

In [56]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'can', 'mustn', 'any', 'from', "you'll", 'below', 'shan', "wouldn't", "haven't", 'should', 'other', 'both', 'hasn', "it's", 'we', 'too', 'very', 'mightn', 'needn', 'before', 'about', "couldn't", 'your', 'him', 'each', 'down', 'between', "hasn't", 'wasn', "you're", "hadn't", 'own', 'me', 'ma', "should've", 'himself', 'to', "you've", 'do', 'ours', 'will', 'itself', 'didn', 'there', 'all', "isn't", 'was', 'some', 'doing', "doesn't", 'with', 'a', 'under', 'they', 'aren', 'after', 'he', 'so', 'hers', 'our', 'isn', 'but', 'yourselves', 'won', 'only', 'same', 'this', 'off', "won't", 'hadn', 'most', 'those', 'were', 'during', 'theirs', 'just', "wasn't", 'an', "aren't", 'd', 'while', "you'd", "don't", 'in', 'his', 'few', 'such', 'at', 'herself', 'further', 'for', 'being', 'of', 'who', 'by', 'ain', 'up', 'i', 'than', 'have', 'over', 'out', 'its', 'into', 'are', 'nor', 'if', 'm', 'doesn', 'above', 're', "mustn't", 've', 'my', 'yours', 'had', 'how', "weren't", 'on', 'until', "didn't", 'o', 'would

In [57]:
tokens_clean = [e for e in tokens if e not in stop_words]
tokens_clean

['What', 'best', 'time', 'year', 'learn', 'Python']

In [37]:
tokens_clean_compound = [sentimentAnalysis(i) for i in tokens_clean]
tokens_clean_compound

[0.0, 0.0, -0.5423, 0.0, 0.6249, 0.0, 0.0]

In [38]:
hola = [(i, j) for i, j in zip(tokens_clean, tokens_clean_compound)]
hola

[('Last', 0.0),
 ('weekend', 0.0),
 ('horrible', -0.5423),
 ('project', 0.0),
 ('great', 0.6249),
 ('weekend', 0.0),
 ('going', 0.0)]

In [39]:

nlp = en_core_web_sm.load()

/usr/local/lib/python3.9/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.3). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [53]:
nlp = spacy.load("en_core_web_sm")
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Autonomous amod cars NOUN []
cars nsubj shift VERB [Autonomous]
shift ROOT shift VERB [cars, liability, toward]
insurance compound liability NOUN []
liability dobj shift VERB [insurance]
toward prep shift VERB [manufacturers]
manufacturers pobj toward ADP []


In [52]:
text = "When Sebastian Thrun started working on self-driving cars at Google in 2007, few people outside of the company took him seriously."

nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
displacy.serve(doc, style="ent")


Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


In [44]:
ironhack_data = "Data prework: Before the course. Dive into the world of data. Once you become an official Ironhack student, you will be given 60 hours of online learning material focusing on command line, Git, Python, MySQL, and statistical analysis. To get you ready for learning , this essential prework will familiarize you with the basics of programming and statistics necessary for the bootcamp. Although you will learn these topics remotely before you arrive at your chosen campus, you won’t be far away from the resources of the Ironhack community. Make use of our Slack channel to connect with your peers and get help from expert academic staff. At the end of your prework, you’ll be ready for the fast pace on campus!"

## 4.3. Audio input

In [ ]:

def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        audio = r.listen(source)
        said = ""

        try:
            said = r.recognize_google(audio)
            print(said)
        except Exception as e:
            print("Exception: " + str(e))
    return said
get_audio()